In [1]:
# Import Counter object
from collections import Counter
# Import pandas for reading data
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Handling data imbalancing
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# Import train_test_split function
from sklearn.model_selection import train_test_split
# Import Keras 
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint

In [2]:
# Dealing with segmented data
#"""select count(1) from "AI_LAGORTA"."CAMP_AUG15_Y_CBM1""""

from sqlalchemy import create_engine
import cx_Oracle


host='192.168.113.80'
port=1521
sid='bbidb'
user='system'
password='oraclebbi'
sid = cx_Oracle.makedsn(host, port, sid=sid)

def oracle_connect(host=host, sid=sid, user=user, password=password):
    cstr = 'oracle://{user}:{password}@{sid}/'.format (
        user = user ,
        password = password ,
        sid = sid
    )

    engine = create_engine (
        cstr ,
        convert_unicode = False ,
        pool_recycle = 10 ,
        pool_size = 50 ,
        echo =False
    )

    return engine.connect()

In [3]:
# retrieve database table
con = oracle_connect()
df = pd.read_sql_query('SELECT * FROM AI_LAGORTA.CAMP_AUG15_Y_CBM2', con = con)
df.columns, len(df.columns)

(Index(['pk_msisdn_id', 'n_accept_optin', 'n_accept_optin_voice_local',
        'n_count_top_ups_last_1m', 'n_data_nat_mb_avg3', 'n_months_acc',
        'n_months_acc_optin_data', 'n_months_pr_optin',
        'n_months_pr_optin_intl', 'n_months_pr_rew_intl',
        'n_months_pr_rew_onnet', 'n_prop_optin_voice_local',
        'n_reward_voice_local', 'n_value_top_ups_last_1m',
        'n_voice_intl_rev_avg3', 'n_voice_intl_secs_avg3',
        'n_voice_off_net_rev_avg3', 'n_voice_on_net_rev_avg3', 'total_voice',
        'v_active_sum', 'v_customer_value$', 'nationality_group', 'target_all'],
       dtype='object'), 23)

In [4]:
# handling ordinal data
v_customer_value_enc = OrdinalEncoder ( categories = [ [ 'LVC' , 'HVC' ] ] )
nationality_group_enc = OrdinalEncoder ( categories = [ [ 'Low' , 'Med' , "High" ] ] )
df [ 'v_customer_value$' ] = v_customer_value_enc.fit_transform (df [ 'v_customer_value$' ].values.reshape ( -1 , 1 ) )
df [ 'nationality_group' ] = nationality_group_enc.fit_transform (df [ 'nationality_group' ].values.reshape ( -1 , 1 ) )
# separate predictors and target
X = df.drop(['pk_msisdn_id', 'target_all'], axis=1)
y = df['target_all'].values

In [5]:
# split dataset into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y) # 70% training and 30% test


In [6]:
len(y_train), len(y_test)

(16672, 7146)

In [7]:
# Training class count
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)
percent = (counts / len(y_train)) * 100
print(np.asarray((unique, percent)).T)

[['0' 96.42514395393475]
 ['1' 3.574856046065259]]


In [8]:
# Testing class count
unique, counts = np.unique(y_test, return_counts=True)
percent = counts 
print(np.asarray((unique, percent)).T)

[['0' 6891]
 ['1' 255]]


In [9]:
# Handling data imbalancing
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Original dataset shape %s' % Counter(y_train))
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'0': 16076, '1': 596})
Resampled dataset shape Counter({'0': 16076, '1': 16076})


In [10]:
input_nodes = (X_res.shape [ 1 ] ,)

## Model building

# model setup
nn = Sequential ( )

# Add input layer
nn.add ( Dense ( 32 , activation = 'relu' , input_shape = input_nodes ) )

# Add two hidden layers of dimension 100
nn.add ( Dense ( 1000 , activation = 'relu' ) )
nn.add ( Dense ( 1000 , activation = 'relu' ) )
# nn.add ( Dense ( 800 , activation = 'relu' ) )
# nn.add ( Dense ( 800 , activation = 'relu' ) )


# Adding the output layer
nn.add ( Dense ( 2 , activation = 'softmax' ) )

# Model compilation
nn.compile ( optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = [ 'accuracy' ] )

# Model fitting
nn.fit ( X_res, y_res, epochs = 50 )
# y_pred_nn = nn.predict_classes ( X_test )


Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/50
32152/32152 [==============================] - 28s 876us/sample - loss: 7.8131 - acc: 0.5144
Epoch 2/50
32152/32152 [==============================] - 29s 901us/sample - loss: 8.0569 - acc: 0.5000
Epoch 3/50
32152/32152 [==============================] - 28s 882us/sample - loss: 7.4079 - acc: 0.5402
Epoch 4/50
32152/32152 [==============================] - 29s 902us/sample - loss: 7.2309 - acc: 0.5512
Epoch 5/50
32152/32152 [==============================] - 30s 918us/sample - loss: 8.0590 - acc: 0.5000
Epoch 6/50
32152/32152 [==============================] - 28s 879us/sample - loss: 8.0590 - acc: 0.5000
Epoch 7/50
32152/32152 [==============================] - 29s 899us/sample - loss: 8.0590 - acc: 0.5000
Epoch 8/50
32152/32152 [==============================] - 29s 902us/sample - loss: 8.0590 - acc: 0.5000
Epoch 9/50
32152/32152 [==============================] - 30s 918us/sample - loss: 8.0590 - acc:

In [11]:
# Training Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_res.astype('int64'), nn.predict_classes(X_res), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.50      1.00      0.67     16076
     class 1       0.00      0.00      0.00     16076

    accuracy                           0.50     32152
   macro avg       0.25      0.50      0.33     32152
weighted avg       0.25      0.50      0.33     32152



C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# Testing Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_test.astype('int64'), nn.predict_classes(X_test), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.96      1.00      0.98      6891
     class 1       0.00      0.00      0.00       255

    accuracy                           0.96      7146
   macro avg       0.48      0.50      0.49      7146
weighted avg       0.93      0.96      0.95      7146



C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
